In [1]:
import string
from scipy.special import hermite
import numpy as np
import tensorflow as tf
from tensorflow.python.ops.special_math_ops import _einsum_v1 as einsum

num_modes = 2
cutoff = 4
num_points = 1000 

alpha = tf.Variable(
    [
        [ 1.0000000e+00-2.9771756e-09j, -2.2837524e-04-4.8372490e-06j, 2.2475776e-04-3.0911028e-06j, -8.8906148e-08-7.1808154e-10j],
        [ 1.0598104e-04+2.2621527e-05j,  5.0736486e-05+1.8292945e-05j, 7.6297164e-09-1.5074212e-09j, -2.2241665e-08+2.6585192e-09j],
        [ 2.5927270e-04+1.0559565e-07j, -5.2198594e-08+3.0939069e-09j, 3.7299468e-08+6.8964878e-10j, -1.7284450e-11-2.9644757e-13j],
        [ 4.7610726e-08+1.0228970e-08j,  4.2539529e-08+3.9197228e-09j, -5.1839774e-12+9.6739988e-13j, -1.8434941e-12+5.1965905e-14j]
    ]
)

x = tf.linspace(-100, 100, num_points)
dx = x[1] - x[0]

def amplitude(
    x: tf.Tensor,
    alpha: tf.Tensor,
    num_modes: int,
    cutoff: int
) -> tf.Tensor:

    num_points = x.shape[0]

    einsum_rule = ''.join(
        [string.ascii_lowercase[: num_modes]] \
        + [',' + string.ascii_lowercase[i] + string.ascii_lowercase[num_modes+i] for i in range(num_modes)] \
        + ['->'] + [string.ascii_lowercase[num_modes+i] for i in range(num_modes)]
    )

    hermite_tensor = tf.Variable(np.zeros(shape=(cutoff, num_points)), dtype=tf.complex128)

    for n in range(cutoff):

        h = hermite(n)
        wave = tf.Variable(h(x)) * tf.exp(-x**2 / 2)/ (np.sqrt(np.sqrt(np.pi)*(2**n)*np.math.factorial(n)))
        wave = tf.cast(wave, tf.complex128)
        hermite_tensor[n].assign(wave)

    amp = einsum(einsum_rule, alpha, *(hermite_tensor for _ in range(num_modes)))

    return amp

def quadratures_densities(
    x: tf.Tensor,
    alpha: tf.Tensor,
    num_modes: int,
    cutoff: int
) -> tf.Tensor:

    amp = amplitude(x, alpha, num_modes, cutoff)
    density = tf.abs(amp)**2

    return density  

In [2]:
rho = quadratures_densities(x, alpha, num_modes, cutoff)

tf.einsum('ab,a,b->', dx**num_modes * rho, x, x)

<tf.Tensor: shape=(), dtype=float64, numpy=5.0746830160629806e-05>

In [13]:
np.einsum('m,n->mn', x, x)


array([[ 10000.        ,   9979.97997998,   9959.95995996, ...,
         -9959.95995996,  -9979.97997998, -10000.        ],
       [  9979.97997998,   9960.00004008,   9940.02010018, ...,
         -9940.02010018,  -9960.00004008,  -9979.97997998],
       [  9959.95995996,   9940.02010018,   9920.0802404 , ...,
         -9920.0802404 ,  -9940.02010018,  -9959.95995996],
       ...,
       [ -9959.95995996,  -9940.02010018,  -9920.0802404 , ...,
          9920.0802404 ,   9940.02010018,   9959.95995996],
       [ -9979.97997998,  -9960.00004008,  -9940.02010018, ...,
          9940.02010018,   9960.00004008,   9979.97997998],
       [-10000.        ,  -9979.97997998,  -9959.95995996, ...,
          9959.95995996,   9979.97997998,  10000.        ]])

In [10]:
np.fromfunction(lambda i, j: j, (3, 3), dtype=float) 

array([[0., 1., 2.],
       [0., 1., 2.],
       [0., 1., 2.]])

In [18]:
import numpy as np

x = np.array([2, 3])


In [19]:
L = x.shape[0]
term01 = np.einsum('a,b->ab', x, x).reshape((L,1,1,L,1,1)).repeat(L,1).repeat(L,2).repeat(L,4).repeat(L,5)
term02 = np.einsum('a,b->ab', x, x).reshape((1,L,1,1,L,1)).repeat(L,0).repeat(L,2).repeat(L,3).repeat(L,5)
term03 = np.einsum('a,b->ab', x, x).reshape((1,1,L,1,1,L)).repeat(L,0).repeat(L,1).repeat(L,3).repeat(L,4)

In [24]:
import tensorflow as tf
x = tf.linspace(0, 1, 3)
L = x.shape[0]

In [25]:
tf.reshape(x, (L,1,1,L,1,1))

InvalidArgumentError: Input to reshape is a tensor with 3 values, but the requested shape has 9 [Op:Reshape]

In [28]:
term01 = tf.repeat(tf.repeat(tf.repeat(tf.repeat(tf.einsum('a,b->ab', x, x)[:,tf.newaxis,tf.newaxis,:,tf.newaxis,tf.newaxis], L, 1), L, 2), L, 4), L, 5)
term02 = tf.repeat(tf.repeat(tf.repeat(tf.repeat(tf.einsum('a,b->ab', x, x)[tf.newaxis,:,tf.newaxis,tf.newaxis,:,tf.newaxis], L, 0), L, 2), L, 3), L, 5)
term03 = tf.repeat(tf.repeat(tf.repeat(tf.repeat(tf.einsum('a,b->ab', x, x)[tf.newaxis,tf.newaxis,:,tf.newaxis,tf.newaxis,:], L, 0), L, 1), L, 3), L, 4)